In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hddm 
import kabuki
import os
from kabuki.analyze import gelman_rubin
from joblib import Parallel,delayed

In [7]:
np.random.seed(1)

In [8]:
def ms1(id, df=None, samples=None, burn=None, thin=1,save_name='ms1'):
    '''
    id: the id of cpu thread
    df: the input data
    samples: number of samples for MCMC
    burn: number of burn-in of MCMC
    thin: number of thin
    save_name: prefix of file name when saving the model object
    '''
    print('running chain {:d} for model {}'.format(id, save_name))
    # the database file
    dbname = save_name + '_chain_%i.db'%id
    # the model object store here
    mname = save_name + '_chain_%i'%id

    v_reg = {'model':'v~cpp*coherence','link_func':lambda x:x}
    t_reg = {'model':'t~1+prior','link_func':lambda x:x}
    
    reg_descr = [v_reg,t_reg]
    m = hddm.HDDMRegressor(df, 
                           reg_descr, 
                           group_only_regressors=False,
                           keep_regressor_trace=True
                          )
    # find the starting point
    m.find_starting_values()
    # MCMC sample
    m.sample(samples, 
             burn=burn, 
             thin=thin, 
             dbname=dbname,
             db='pickle')
    # save
    m.save(mname)

    return m

In [9]:
def simulate_data(nr_trials=288, n_subj=16, effect_size=0.5, cpp_type="trialwise", bins=4):
    """
    Simulate experimental data and generate reaction time (RT) and response data 
    for trial-wise, bin-wise, or condition-wise CPP.

    Parameters:
        nr_trials (int): Number of trials per subject.
        n_subj (int): Number of subjects.
        effect_size (float): Effect size for the simulation.
        cpp_type (str): Type of CPP ('trialwise', 'binwise', or 'conditionwise').
        bins (int): Number of bins for bin-wise CPP calculation.

    Returns:
        pd.DataFrame: A dataframe containing the simulated data.
    """
    # Generate CPP data for each subject and trial
    cpp = np.random.normal(0, 1, (n_subj, nr_trials))
    coherence = np.concatenate((np.ones(int(nr_trials/2)), np.zeros(int(nr_trials/2))))
    prior = np.concatenate((np.ones(int(nr_trials/4)), np.zeros(int(nr_trials/4)),np.ones(int(nr_trials/4)), np.zeros(int(nr_trials/4))))
    effct_size_subj = np.random.normal(effect_size, 0.1, n_subj)
    effct_size_subj_int = np.random.normal(0, 0.1, n_subj)
    effct_size_subj_coh = np.random.normal(1, 0.4, n_subj)
    effct_size_subj_pri = np.random.normal(-0.02, 0.01, n_subj)

    # Initialize storage for all simulated data
    sim_data = pd.DataFrame()

    # Generate data for each subject
    for j in range(n_subj):
        param = pd.DataFrame()

        # Trial-specific v values
        param['v'] = cpp[j] * effct_size_subj[j] + 3 + effct_size_subj_int[j]*cpp[j]*coherence + effct_size_subj_coh[j]*coherence

        # Fixed parameters: t, a, z
        param['t'] = 0.3 + effct_size_subj_pri[j]*prior
        param['a'] = 1
        param['z'] = 0.5
        param.index = param.index.astype(str)
        param = param.to_dict(orient='index')

        # Generate RT and response data using HDDM
        data, params = hddm.generate.gen_rand_data(param, subjs=1, size=1)

        # Add CPP information
        data['cpp'] = cpp[j]

        if cpp_type == "binwise":
            # Calculate bin-wise CPP
            data['cpp_bin'] = pd.cut(data['cpp'], bins=bins, labels=range(1, bins + 1))
            data['cpp'] = data.groupby('cpp_bin')['cpp'].transform('mean')
        elif cpp_type == "conditionwise":
            # Calculate condition-wise CPP (average across trials)
            data['cpp'] = np.mean(cpp[j])

        # Add subject index
        data['subj_idx'] = j
        data['coherence'] = coherence
        data['prior'] = prior
        

        # Concatenate data for all subjects
        sim_data = pd.concat([sim_data, data], ignore_index=True)

    return sim_data


In [10]:
# Function for generating data and running ms1 simulation
def run_simulation_and_ms1(ms1_function, cpp_type, n_jobs=4, n_iterations=0, nr_trials=288, n_subj=16, effect_size=0.5, bins=4, samples=8000, burn=2000, thin=2, save_name_prefix='ms1test'):
    """
    Generate data (trial-wise, bin-wise, or condition-wise) and run ms1 simulations in parallel.

    Parameters:
        ms1_function (function): The ms1 function to execute.
        cpp_type (str): Type of CPP data ('trialwise', 'binwise', or 'conditionwise').
        n_jobs (int): Number of parallel jobs.
        n_iterations (int): Number of iterations to run the ms1 function (default: 100).
        nr_trials (int): Number of trials per subject.
        n_subj (int): Number of subjects.
        effect_size (float): Effect size for simulation.
        bins (int): Number of bins (only for bin-wise CPP).
        samples (int): Number of samples for ms1.
        burn (int): Number of burn-in samples for ms1.
        thin (int): Thinning interval for ms1.
        save_name_prefix (str): Prefix for saved file names.

    Returns:
        list: Results from the ms1 simulation for all iterations.
    """
    # Generate data based on CPP type
    sim_data = simulate_data(nr_trials=nr_trials, n_subj=n_subj, effect_size=effect_size, cpp_type=cpp_type, bins=bins)
    
    # Run ms1 in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(ms1_function)(
            id=i,
            df=sim_data,
            samples=samples,
            burn=burn,
            thin=thin,
            save_name=f"{save_name_prefix}_{cpp_type}_{n_iterations:03d}"  # Add zero-padded numbering
        ) for i in range(n_jobs)
    )
    return results



In [11]:

# Run parallel simulations for 100 iterations of trial-wise data
cpp_types = ["trialwise", 
             #"binwise", 
             #"conditionwise"
            ]
n_jobs = 4
for effect_size in [0.2]:
    for i in range(4):
        results = Parallel(n_jobs=len(cpp_types)*n_jobs)(delayed(run_simulation_and_ms1)(
            ms1_function=ms1, 
            cpp_type=cpp_type, 
            n_jobs=n_jobs, 
            n_iterations = i, 
            nr_trials=288, 
            n_subj=16, 
            effect_size=effect_size, 
            samples=6000, 
            burn=3000, 
            thin=2,
            save_name_prefix='tempdata/trialwise/ms1full'
        ) for cpp_type in cpp_types)
        print(np.sum(np.array(list(gelman_rubin(results[0]).values()))>1.1))
        print(gelman_rubin(results[0]))
        # combine the model
        m1 = kabuki.utils.concat_models(results[0])
        m1.print_stats()


running chain 0 for model tempdata/trialwise/ms1full_trialwise_000
running chain 1 for model tempdata/trialwise/ms1full_trialwise_000
running chain 2 for model tempdata/trialwise/ms1full_trialwise_000
running chain 3 for model tempdata/trialwise/ms1full_trialwise_000
Adding these covariates:
['v_Intercept', 'v_cpp', 'v_coherence', 'v_cpp:coherence']
Adding these covariates:
['t_Intercept', 't_prior']
Adding these covariates:
['v_Intercept', 'v_cpp', 'v_coherence', 'v_cpp:coherence']
Adding these covariates:
['v_Intercept', 'v_cpp', 'v_coherence', 'v_cpp:coherence']
Adding these covariates:
['t_Intercept', 't_prior']
Adding these covariates:
['t_Intercept', 't_prior']
Adding these covariates:
['v_Intercept', 'v_cpp', 'v_coherence', 'v_cpp:coherence']
Adding these covariates:
['t_Intercept', 't_prior']


/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


[                  0%                  ] 2 of 6000 complete in 0.7 sec
[                  0%                  ] 2 of 6000 complete in 1.0 sec
[                  0%                  ] 3 of 6000 complete in 1.6 sec
[                  0%                  ] 3 of 6000 complete in 1.6 sec
[                  0%                  ] 4 of 6000 complete in 3.0 sec
[                  0%                  ] 2 of 6000 complete in 2.5 sec
[                  0%                  ] 4 of 6000 complete in 3.4 sec
[                  0%                  ] 2 of 6000 complete in 2.4 sec
[                  0%                  ] 5 of 6000 complete in 4.6 sec
[                  0%                  ] 3 of 6000 complete in 4.2 sec
[                  0%                  ] 5 of 6000 complete in 4.9 sec
[                  0%                  ] 3 of 6000 complete in 4.5 sec
[                  0%                  ] 6 of 6000 complete in 6.0 sec
[                  0%                  ] 4 of 6000 complete in 5.7 sec
[     

KeyboardInterrupt: 

8